## Read the csv

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('/content/NYT_Dataset.csv')

In [3]:
data.head()

,Unnamed: 0,ID,title,topic,abstract,Date,keywords
0,0,nyt://article/178801fe-4679-5f12-985f-8344a86e...,"In Reversal, Pakistan Welcomes Outside Help Wi...",Foreign,Pakistan’s ambassador to the U.S. said his gov...,2008-01-01 05:00:00+00:00,['Assassinations and Attempted Assassinations'...
1,1,nyt://article/21acedcb-a7f6-5131-99cf-d3a47e33...,Fighting Intensifies After Election in Kenya,Foreign,"Kenya sank deeper into trouble, with a curfew ...",2008-01-01 05:00:00+00:00,"['Kenya', 'Demonstrations and Riots', 'Odinga,..."
2,2,nyt://article/357b5429-a9f8-5d33-a5eb-c013a201...,Israel: Olmert Curbs Settlements,Foreign,Prime Minister Ehud Olmert has sent a letter t...,2008-01-01 05:00:00+00:00,['West Bank']
3,3,nyt://article/619ca4ea-50e4-59e4-97bb-f206502c...,Gay Muslims Pack a Dance Floor of Their Own,Foreign,The monthly club night known as Gayhane is an ...,2008-01-01 05:00:00+00:00,"['Homosexuality', 'Islam', 'IMMIGRATION AND RE..."
4,4,nyt://article/73c49a5a-bcf1-5b8f-a15a-98d29003...,Iraqi Revelers Embrace the New Year,Foreign,"But even as partygoers embraced the New Year, ...",2008-01-01 05:00:00+00:00,"['ARMAMENT, DEFENSE AND MILITARY FORCES', 'Iraq']"


In [4]:
data = data.drop(['Unnamed: 0','ID','topic','Date','keywords'],axis=1)

In [5]:
data.head()

,title,abstract
0,"In Reversal, Pakistan Welcomes Outside Help Wi...",Pakistan’s ambassador to the U.S. said his gov...
1,Fighting Intensifies After Election in Kenya,"Kenya sank deeper into trouble, with a curfew ..."
2,Israel: Olmert Curbs Settlements,Prime Minister Ehud Olmert has sent a letter t...
3,Gay Muslims Pack a Dance Floor of Their Own,The monthly club night known as Gayhane is an ...
4,Iraqi Revelers Embrace the New Year,"But even as partygoers embraced the New Year, ..."


In [8]:
data['abstract'][0]

'Pakistan’s ambassador to the U.S. said his government would not endorse a separate inquiry modeled after one carried out by the U.N. after the assassination of Rafik Hariri of Lebanon in 2005.'

In [6]:
data.isna().sum()

title         2
abstract    582
dtype: int64

In [7]:
data=data.dropna()

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 105923 entries, 0 to 106505
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   title     105923 non-null  object
 1   abstract  105923 non-null  object
dtypes: object(2)
memory usage: 6.5+ MB


In [10]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


## Using Huggingface pre-trained model and tokenizer BART-large by Facebook

In [ ]:
from datasets import Dataset
from transformers import BartForConditionalGeneration, BartTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
dataset = Dataset.from_pandas(data)

# Tokenize the dataset
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")

def preprocess_function(examples):
    inputs = [doc for doc in examples["abstract"]]
    targets = [doc for doc in examples["title"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding="max_length", return_tensors="pt")

    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length", return_tensors="pt").input_ids
    labels = [[(label if label != tokenizer.pad_token_id else -100) for label in labels_example] for labels_example in labels]
    model_inputs["labels"] = labels

    return model_inputs

In [ ]:
!pip install transformers[torch] accelerate>=0.21.0

In [ ]:
import random
data_random = data.sample(frac=0.01)

In [ ]:
data_random.head()

,title,abstract
29712,"Egyptian Actor Insulted Islam, a Court Finds","A court’s ruling against Adel Imam, Egypt’s mo..."
41541,North Korea Accuses Captive U.S. Veteran of Wa...,The country’s official news agency says Merril...
38579,"Hans Hass, Early Undersea Explorer, Dies at 94","Dr. Hass, a marine scientist from Austria with..."
43145,Indonesia Takes Aim at Australia Over Spying o...,At a news conference with Secretary of State J...
18028,Mitt Romney Campaigns in New Hampshire,"Mr. Romney's regional appeal, bolstered by hea..."


In [ ]:
data_random.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1059 entries, 29712 to 40308
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     1059 non-null   object
 1   abstract  1059 non-null   object
dtypes: object(2)
memory usage: 24.8+ KB


## only 0.01% of the data for testing

## Splitting and Tokenizing

In [ ]:
from sklearn.model_selection import train_test_split

# Split the tokenized dataset into training and validation sets
train_data_random, val_data_random = train_test_split(data_random, test_size=0.15)

# Create Hugging Face Dataset from the split pandas DataFrames
train_dataset_small = Dataset.from_pandas(train_data_random)
val_dataset_small = Dataset.from_pandas(val_data_random)

In [ ]:
tokenized_train_dataset_small = train_dataset_small.map(preprocess_function, batched=True)
tokenized_val_dataset_small = val_dataset_small.map(preprocess_function, batched=True)

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/159 [00:00<?, ? examples/s]

## Model Training

In [ ]:
from datasets import load_metric

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./small-test",  # Define the directory where model checkpoints and logs will be saved
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    logging_steps=100,
    save_steps=100,
    eval_steps=100,
    warmup_steps=200,
    num_train_epochs=3,
    overwrite_output_dir=True,
    save_total_limit=3,  # Specify the compute_metrics function
)
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")

# Define Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset_small,
    eval_dataset=tokenized_val_dataset_small,
    tokenizer=tokenizer,
)

# Start training
trainer.train()

Step,Training Loss,Validation Loss
100,3.733200,2.848417
200,3.362600,2.945534
300,3.686700,3.101218
400,3.583700,3.076187
500,3.033200,3.002229
600,2.549400,2.995589
700,2.500900,2.961797
800,2.429300,3.166981
900,2.336400,2.975675
1000,1.475100,3.264713


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file

TrainOutput(global_step=1350, training_loss=2.5029998948838976, metrics={'train_runtime': 3030.2612, 'train_samples_per_second': 0.891, 'train_steps_per_second': 0.446, 'total_flos': 5851182425702400.0, 'train_loss': 2.5029998948838976, 'epoch': 3.0})

In [ ]:
eval_results = trainer.evaluate(eval_dataset=tokenized_val_dataset_small)
print("Evaluation results:", eval_results)

Evaluation results: {'eval_loss': 3.080737590789795, 'eval_runtime': 7.6568, 'eval_samples_per_second': 20.766, 'eval_steps_per_second': 10.448, 'epoch': 3.0}


## Example abstract and title predicted

In [ ]:
your_abstract = "Prime Minister David Cameron said that he had acted to protect Britain’s interests and that he had not consigned the country to Europe’s sidelines."

inputs = tokenizer(your_abstract, return_tensors="pt", max_length=1024, truncation=True)

inputs = {k: v.to(model.device) for k, v in inputs.items()}

generated_title = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_length=128, num_beams=4, early_stopping=True)

decoded_title = tokenizer.decode(generated_title[0], skip_special_tokens=True)

# Print the generated title
print("Generated Title:", decoded_title)


Generated Title: Cameron Says He Stayed in Europe to Protect Britain’s Interests


In [ ]:
data_random['title'][27055]

'Cameron Says His Veto on Europe Treaty Protects Britain'

## Using OpenAI API (GPT Turbo 3.5)

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00


In [13]:
# Setting up the key
import openai
import pandas as pd

openai.api_key = "sk-pNLX5lEPUr65t42dtzJXgdEMT3hjhbvfcbygbBlbkFJ4tQKRAftLjddukrmabbjnjnjnhbhb"

In [14]:
abstract = 'Prime Minister David Cameron said that he had acted to protect Britain’s interests and that he had not consigned the country to Europe’s sidelines.'

In [16]:
# Define the custom abstract
custom_abstract = """
This study investigates the impact of environmental factors on the growth and development of plant species in tropical rainforests. We conducted field experiments and collected data on various climatic conditions, soil composition, and the presence of symbiotic organisms. The findings suggest that temperature, humidity, and nutrient availability play a crucial role in determining the growth rates and overall health of the studied plant species. Additionally, the presence of certain fungi and bacteria in the soil had a significant influence on plant growth and resilience to environmental stressors.
"""

def generate_title(abstract):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that generates titles for news abstracts."},
            {"role": "user", "content": f"Abstract: {abstract}\n\nGenerate a concise title for the given abstract."}
        ],
        temperature=0.7,
    )

    title = response.choices[0].message.content.strip()
    return title

# Call the function with the custom abstract
predicted_title = generate_title(custom_abstract)
print(f"Predicted Title: {predicted_title}")

Predicted Title: Title: Environmental Factors Shaping Plant Growth in Tropical Rainforests


In [17]:
data.head()

,title,abstract
0,"In Reversal, Pakistan Welcomes Outside Help Wi...",Pakistan’s ambassador to the U.S. said his gov...
1,Fighting Intensifies After Election in Kenya,"Kenya sank deeper into trouble, with a curfew ..."
2,Israel: Olmert Curbs Settlements,Prime Minister Ehud Olmert has sent a letter t...
3,Gay Muslims Pack a Dance Floor of Their Own,The monthly club night known as Gayhane is an ...
4,Iraqi Revelers Embrace the New Year,"But even as partygoers embraced the New Year, ..."


In [18]:
data_rand = data.sample(frac=0.01)

In [19]:
data_rand.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1059 entries, 101691 to 97348
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     1059 non-null   object
 1   abstract  1059 non-null   object
dtypes: object(2)
memory usage: 24.8+ KB


## Predicting titles using GPT API with the abstract columns : reason being that most of the keywords are already in the abstract and plus GPT being a powerful model if given an access to the abstract and keywords would easily edge out and outperform the other models

In [20]:
def generate_title(abstract):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that generates titles for abstracts."},
            {"role": "user", "content": f"Abstract: {abstract}\n\nGenerate a concise title for the given abstract."}
        ],
        temperature=0.7,
    )

    title = response.choices[0].message.content.strip()
    return title

data_rand['predicted_title']=data_rand['abstract'].apply(generate_title)


In [21]:
data_rand

,title,abstract,predicted_title
101691,California Election Results: 17th Congressiona...,See full results and maps from the California ...,"""California Election: Full Results and Maps Re..."
47292,Q. and A.: Rana Mitter on the Legacy of World ...,"The Oxford historian says that, unlike in Euro...","Title: ""Divergent Perspectives on War: The Imp..."
5507,Bangladesh Delays Vote,The army-backed caretaker government of Bangla...,Title: Postponement of Bangladesh Elections by...
10820,Rare Uganda Unrest Centers on Local King,"Bands of men burned tires, looted shops and ba...","""Violent Unrest in Kampala: Men Rioting and Cl..."
15848,Taliban Slay Elders Over Aid Money,"Tribal elders, particularly those in Kandahar,...",Title: Targeting Tribal Elders in Kandahar: Ta...
...,...,...,...
26250,Austerity Faces Test as Greeks Question Their ...,The euro crisis has finally touched real polit...,"Title: The Greek Dilemma: Europe, Currency, an..."
92659,Wuhan Rounds Up the Infected as Death Toll in ...,Health officials are focusing on faster diagno...,"""Urgency in Coronavirus Diagnosis: Chinese Doc..."
64377,China Urging Afghanistan to Restart Peace Talk...,A statement from the Chinese Foreign Ministry ...,Title: China's Commitment to the Peace Process
89581,Iran’s Rouhani Says No Talks With Trump Until ...,President Emmanuel Macron of France is trying ...,"""Efforts of Presidents Macron and Rouhani to F..."


In [22]:
data_rand['predicted_title'] = data_rand['predicted_title'].apply(lambda x: x.replace('Title: ', '') if x.startswith('Title: ') else x)

# Display the modified DataFrame
data_rand

,title,abstract,predicted_title
101691,California Election Results: 17th Congressiona...,See full results and maps from the California ...,"""California Election: Full Results and Maps Re..."
47292,Q. and A.: Rana Mitter on the Legacy of World ...,"The Oxford historian says that, unlike in Euro...","""Divergent Perspectives on War: The Impact on ..."
5507,Bangladesh Delays Vote,The army-backed caretaker government of Bangla...,Postponement of Bangladesh Elections by Careta...
10820,Rare Uganda Unrest Centers on Local King,"Bands of men burned tires, looted shops and ba...","""Violent Unrest in Kampala: Men Rioting and Cl..."
15848,Taliban Slay Elders Over Aid Money,"Tribal elders, particularly those in Kandahar,...",Targeting Tribal Elders in Kandahar: Taliban's...
...,...,...,...
26250,Austerity Faces Test as Greeks Question Their ...,The euro crisis has finally touched real polit...,"The Greek Dilemma: Europe, Currency, and Sover..."
92659,Wuhan Rounds Up the Infected as Death Toll in ...,Health officials are focusing on faster diagno...,"""Urgency in Coronavirus Diagnosis: Chinese Doc..."
64377,China Urging Afghanistan to Restart Peace Talk...,A statement from the Chinese Foreign Ministry ...,China's Commitment to the Peace Process
89581,Iran’s Rouhani Says No Talks With Trump Until ...,President Emmanuel Macron of France is trying ...,"""Efforts of Presidents Macron and Rouhani to F..."


## 15 examples of the generated titles and the actual predicted titles from the abstract

In [23]:
random_rows = data_rand.sample(n=15, random_state=330)  # Adjust the random state for reproducibility

# Display the selected random titles and predicted titles
for index, row in random_rows.iterrows():
    print("Title:", row['title'])
    print("Predicted Title:", row['predicted_title'])
    print("\n")

Title: Britain: Conservatives Back Off Treaty Vote
Predicted Title: David Cameron Declines Lisbon Treaty Referendum Promise


Title: Obama Urges Democrats to ‘Chill Out’ About 2020 Presidential Field
Predicted Title: Obama Urges Democratic Unity Against Trump


Title: Their Protest Helped End the Draft. 50 Years Later, It’s Still Controversial.
Predicted Title: Catholic Activists Burn Draft Files in Maryland Parking Lot


Title: Refugees’ Lives ‘Stripped of Meaning’ by Detention, Australian Author Says
Predicted Title: Extreme Suffering of Refugees on Nauru and Manus


Title: Putin Does Not Expect Ties With U.S. to Be Harmed by Snowden Case
Predicted Title: Putin Emphasizes Importance of Bilateral Relations Over Snowden Controversy


Title: Putin and Erdogan Announce Plan for Northeast Syria, Bolstering Russian Influence
Predicted Title: Russia Emerges as a Key Player in the Middle East


Title: Parents of P.O.W. Reveal U.S. Talks on Taliban Swap
Predicted Title: Revelations of Secret 

## BLEU and ROUGE Scores for 0.01% of the data taken into account

In [24]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction


# Extract the abstracts and generated titles as lists
abstracts = data_rand['title'].tolist()
generated_titles = data_rand['predicted_title'].tolist()

# Function to calculate BLEU score
def calculate_bleu_score(reference, hypothesis):
    smoothing_function = SmoothingFunction().method4
    bleu_score = sentence_bleu([reference], hypothesis, smoothing_function=smoothing_function)
    return bleu_score

# Calculate BLEU score for each abstract-title pair
bleu_scores = []
for abstract, title in zip(abstracts, generated_titles):
    bleu_score = calculate_bleu_score(abstract, title)
    bleu_scores.append(bleu_score)

# Calculate the average BLEU score
avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
print(f"Average BLEU Score: {avg_bleu_score:.4f}")

Average BLEU Score: 0.2374


In [25]:
!pip install rouge

In [28]:
import pandas as pd
from rouge import Rouge

# Extract the abstracts and generated titles as lists
abstracts = data_rand['title'].tolist()
generated_titles = data_rand['predicted_title'].tolist()

# Function to calculate ROUGE-L score
def calculate_rouge_score(reference, hypothesis):
    rouge = Rouge()
    scores = rouge.get_scores(hypothesis, reference)
    rouge_l_score = scores[0]['rouge-l']['f']
    return rouge_l_score

# Calculate ROUGE-L score for each abstract-title pair
rouge_scores = []
for abstract, title in zip(abstracts, generated_titles):
    rouge_score = calculate_rouge_score(abstract, title)
    rouge_scores.append(rouge_score)

# Calculate the average ROUGE-L score
avg_rouge_score = sum(rouge_scores) / len(rouge_scores)
print(f"Average ROUGE-L Score: {avg_rouge_score:.4f}")

Average ROUGE-L Score: 0.1263


## Trying the GPT Turbo API on over 5K abstracts

In [ ]:
data_random_1 = data.sample(frac=0.05)

In [ ]:
data_random_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5296 entries, 77144 to 44439
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     5296 non-null   object
 1   abstract  5296 non-null   object
dtypes: object(2)
memory usage: 124.1+ KB


In [ ]:
data_random_1.head()

,title,abstract
77144,Where Harvey Hit Hardest Up and Down the Texas...,"Over 150,000 properties in Texas have been aff..."
25614,A High-Tech War Against Slights to a Centuries...,A “war room” of technicians weeds out sites de...
97131,A shift on election briefings to Congress coul...,The elimination of in-person election security...
82340,"Andrew Brunson, U.S. Pastor, Moved to House Ar...",The U.S. said the move was ‘not enough.’ Mr. B...
62902,Chris Christie on the Death Penalty,Read the latest updates from Mr. Christie.


In [ ]:
def generate_title(abstract):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that generates titles for abstracts."},
            {"role": "user", "content": f"Abstract: {abstract}\n\nGenerate a concise title for the given abstract."}
        ],
        temperature=0.7,
    )

    title = response.choices[0].message.content.strip()
    return title

data_random_1['predicted_title']=data_random_1['abstract'].apply(generate_title)

In [ ]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction


abstracts = data_random_1['title'].tolist()
generated_titles = data_random_1['predicted_title'].tolist()

def calculate_bleu_score(reference, hypothesis):
    smoothing_function = SmoothingFunction().method4
    bleu_score = sentence_bleu([reference], hypothesis, smoothing_function=smoothing_function)
    return bleu_score

bleu_scores = []
for abstract, title in zip(abstracts, generated_titles):
    bleu_score = calculate_bleu_score(abstract, title)
    bleu_scores.append(bleu_score)

# Calculate the average BLEU score
avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
print(f"Average BLEU Score: {avg_bleu_score:.4f}")

Average BLEU Score: 0.2273


In [ ]:
import pandas as pd
from rouge import Rouge

abstracts = data_random_1['title'].tolist()
generated_titles = data_random_1['predicted_title'].tolist()

def calculate_rouge_score(reference, hypothesis):
    rouge = Rouge()
    scores = rouge.get_scores(hypothesis, reference)
    rouge_l_score = scores[0]['rouge-l']['f']
    return rouge_l_score

rouge_scores = []
for abstract, title in zip(abstracts, generated_titles):
    rouge_score = calculate_rouge_score(abstract, title)
    rouge_scores.append(rouge_score)

# Calculate the average ROUGE-L score
avg_rouge_score = sum(rouge_scores) / len(rouge_scores)
print(f"Average ROUGE-L Score: {avg_rouge_score:.4f}")

Average ROUGE-L Score: 0.1178
